# Time-Series Momentum + Volatility Targeting — 2800.HK

- Logic: direction = sign of lookback return; position size scaled by target volatility.
- This is a more ``systematic'' classic strategy because it includes risk scaling.
- Backtest: close-to-close; transaction cost charged on position changes (bps).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import VolTargetMomentumConfig, vol_target_momentum_position

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
TRANSACTION_COST_BP = 2.0

CFG = VolTargetMomentumConfig(
    mom_lookback=60,
    vol_lookback=20,
    target_vol_annual=0.10,
    max_leverage=1.0,
    allow_short=True,
)

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()

pos = vol_target_momentum_position(close, CFG)

next_ret = np.log(close.shift(-1) / close)
simple_ret = (np.exp(next_ret) - 1.0).dropna()

bt_df, stats = equity_and_stats_from_positions(
    simple_return=simple_ret,
    position=pos.loc[simple_ret.index],
    transaction_cost_bp=TRANSACTION_COST_BP,
)

print(pd.Series(stats))

plt.figure(figsize=(12,4))
plt.plot(bt_df["strategy_equity"], label="Vol-target momentum")
plt.plot(bt_df["buyhold_equity"], label="Buy&Hold", linestyle="--")
plt.title(
    f"Vol-Target Momentum | {TICKER} | mom={CFG.mom_lookback}d, vol={CFG.vol_lookback}d, targetVol={CFG.target_vol_annual:.0%}, cost={TRANSACTION_COST_BP:.0f}bps"
)
plt.legend()
plt.grid(alpha=0.3)
plt.show()
